# Model and stuff

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import time
import os
import random

# Image Cleansing

In [13]:
# --- 1. CONFIGURATION (for MLP-Only Model) ---
import torch

CONFIG = {
    # --- MODIFIED: Single model path template ---
    "MODEL_PATH_TPL": "best_translator_mlp_fold_{}.pth",
    "DEVICE": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    
    # --- Training ---
    "EPOCHS": 100, # Your log shows 100
    "PATIENCE": 25,
    
    # --- K-Fold Configuration ---
    "N_FOLDS": 10,  # Your log shows 5
    "RANDOM_STATE": 42,
    
    # --- Architecture/Loss Params (Using your tuned values) ---
    "HIDDEN_FEATURES": 3072, 
    "NUM_BLOCKS": 2,
    "DROPOUT_RATE": 0.3832296399028748,     
    "LR": 1.7749224152998916e-05,             
    "WEIGHT_DECAY": 0.0008723639231133962,
    
    # --- Hybrid Loss Params ---
    "MARGIN": 0.3898481154806984,
    "TEMPERATURE": 0.010879304099273002,
    "HYBRID_ALPHA": 0.0, # Kept at 0
    "MIXUP_ALPHA": 0.23197266859653176,
    "LABEL_SMOOTHING": 0.12,
    
    # --- General Settings ---
    "BATCH_SIZE": 256,
    "ACCUMULATION_STEPS": 2,
}

In [14]:
# --- 10. Data Loading (MODIFIED for Cleaned Data) ---
from sklearn.model_selection import KFold
import numpy as np
import torch

try:
    # --- 1. Load Full (Noisy) Data ---
    full_train_data = np.load("/kaggle/input/aml-challenge-dataset/train.npz", allow_pickle=True)
    all_text_embeddings_full = torch.from_numpy(full_train_data['captions/embeddings']).float()
    all_image_embeddings_full = torch.from_numpy(full_train_data['images/embeddings']).float()
    all_image_ids_full = full_train_data['images/names']
    
    num_images_total = all_image_embeddings_full.shape[0]
    num_captions_per_image = all_text_embeddings_full.shape[0] // num_images_total

    # --- 2. Load the Clean Indices ---
    print("Loading clean indices from '/kaggle/input/clean-dataset/'...")
    clean_caption_indices = np.load("/kaggle/input/clean-dataset/clean_caption_indices.npy")
    # clean_image_indices = np.load("/kaggle/input/clean-dataset/clean_image_indices.npy")
    print(f"Loaded {len(clean_caption_indices)} clean caption indices.")
    
    # --- 3. Filter Embeddings ---
    # We will build our new dataset from the clean CAPTION indices
    # This creates a 1-to-1 (text, image) dataset
    
    print("Filtering embeddings based on clean indices...")
    # Filter text embeddings (e.g., 100,000 x 1024)
    all_text_embeddings = all_text_embeddings_full[clean_caption_indices]
    
    # Find the corresponding image for each clean caption
    # e.g., caption 12 -> image 2 (12 // 5)
    # e.g., caption 14 -> image 2 (14 // 5)
    corresponding_image_indices = clean_caption_indices // num_captions_per_image
    
    # Filter image embeddings (e.g., 100,000 x 1536)
    all_image_embeddings = all_image_embeddings_full[corresponding_image_indices]
    
    print(f"Clean data shape (text): {all_text_embeddings.shape}")
    print(f"Clean data shape (image): {all_image_embeddings.shape}")

    # --- 4. Setup K-Fold for 1-to-1 Pairs ---
    # We now split the PAIRS, not the images
    num_clean_pairs = len(all_text_embeddings)
    pair_indices = np.arange(num_clean_pairs)
    
    kfold = KFold(n_splits=CONFIG['N_FOLDS'], shuffle=True, random_state=CONFIG['RANDOM_STATE'])

    print(f"\nCLEAN Data loaded: {num_clean_pairs} text/image pairs.")
    print(f"KFold splitter created for {num_clean_pairs} pairs.")

except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Make sure 'train.npz' is at '/kaggle/input/train/train/train.npz'")
    print("And your 'clean-dataset' is attached.")

Loading clean indices from '/kaggle/input/clean-dataset/'...
Loaded 123460 clean caption indices.
Filtering embeddings based on clean indices...
Clean data shape (text): torch.Size([123460, 1024])
Clean data shape (image): torch.Size([123460, 1536])

CLEAN Data loaded: 123460 text/image pairs.
KFold splitter created for 123460 pairs.


# Model Architecture

In [17]:
# --- 3. Model Architecture (MODIFIED: MLP-Only) ---
import torch
import torch.nn as nn
import torch.nn.functional as F

# --- 3A. The GeGLU Activation Module (Unchanged) ---
class GeGLU(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.proj = nn.Linear(in_features, out_features * 2)
        self.gelu = nn.GELU()
    def forward(self, x):
        proj_out = self.proj(x)
        a, b = proj_out.chunk(2, dim=-1)
        return self.gelu(a) * b

# --- 3B. The Residual Block with GeGLU (Unchanged) ---
class ResidualBlockGeGLU(nn.Module):
    def __init__(self, features, dropout_rate):
        super().__init__()
        self.block = nn.Sequential(
            GeGLU(features, features), 
            nn.BatchNorm1d(features),
            nn.Dropout(dropout_rate)
        )
    def forward(self, x):
        return x + self.block(x)

# --- 3C. REMOVED TextVariationalEncoder ---
# This class is no longer needed for this architecture.

# --- 3D. The Translator (Your HybridMLP) (Unchanged) ---
# This class is already flexible. 
# 'in_features' will be 1024 (text dim) when we create the model.
class TranslatorMLP(nn.Module):
    def __init__(self, in_features, out_features, hidden_features, num_blocks, dropout_rate):
        super().__init__()
        
        backbone_layers = [
            nn.Linear(in_features, hidden_features, bias=False),
            nn.BatchNorm1d(hidden_features),
            nn.GELU(),
            nn.Dropout(0.5)
        ]
        for _ in range(num_blocks):
            backbone_layers.append(
                ResidualBlockGeGLU(hidden_features, dropout_rate) # Using GeGLU
            )
        self.backbone = nn.Sequential(*backbone_layers)
        
        self.translator_head = nn.Sequential(
            nn.Linear(hidden_features, hidden_features // 2),
            nn.GELU(),
            nn.Linear(hidden_features // 2, out_features) # out_features = 1536
        )

    def forward(self, x):
        shared_representation = self.backbone(x)
        output = self.translator_head(shared_representation)
        # We keep the normalization, as discussed
        return F.normalize(output, p=2, dim=1)

# Helper Functions

In [18]:
# --- 5. Helper Functions (for MLP-Only Model) ---
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# --- 5A. Loss Functions (Unchanged) ---
class PureContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.07, label_smoothing=0.12):
        super().__init__()
        self.temperature = temperature
        self.label_smoothing = label_smoothing # <-- MODIFIED
    def forward(self, pred_norm, target_norm):
        sim_matrix = torch.matmul(pred_norm, target_norm.T) / self.temperature
        labels = torch.arange(pred_norm.size(0), device=pred_norm.device)
        return F.cross_entropy(sim_matrix, labels, label_smoothing=self.label_smoothing)
class TripletLoss(nn.Module):
    def __init__(self, margin=0.2):
        super().__init__()
        self.margin = margin
    def forward(self, text_embeds, img_embeds_norm):
        sim_matrix = text_embeds @ img_embeds_norm.T
        positive_scores = torch.diag(sim_matrix)
        mask = torch.eye(text_embeds.size(0), dtype=torch.bool, device=text_embeds.device)
        sim_matrix_masked = sim_matrix.masked_fill(mask, -float('inf'))
        hard_negative_scores = sim_matrix_masked.max(dim=1)[0]
        return F.relu(self.margin - positive_scores + hard_negative_scores).mean()

# --- 5B. FAST VALIDATION (MODIFIED for single model) ---
# This version only takes 3 arguments
def validation_fn(translator_model, val_loader, criterion_triplet, criterion_pure, alpha, device):
    translator_model.eval()
    val_loss = 0
    with torch.no_grad():
        for text_batch, image_batch in val_loader:
            text_batch, image_batch = text_batch.to(device), image_batch.to(device)
            
            pred_embeddings = translator_model(text_batch)
            target_embeddings = F.normalize(image_batch, p=2, dim=1)
            
            loss_triplet = criterion_triplet(pred_embeddings, target_embeddings)
            loss_pure = criterion_pure(pred_embeddings, target_embeddings)
            hybrid_loss = (alpha * loss_triplet) + ((1 - alpha) * loss_pure)
            
            val_loss += hybrid_loss.item()
            
    return val_loss / len(val_loader)

# --- 5C. MRR CALCULATION (MODIFIED for single model) ---
# This version only takes 3 arguments
def calculate_mrr(translator_model, val_loader, device):
    translator_model.eval()
    all_reciprocal_ranks = []
    with torch.no_grad():
        for text_batch, image_batch in val_loader:
            text_batch = text_batch.to(device)
            image_batch = image_batch.to(device)
            
            pred_embeddings_norm = translator_model(text_batch)
            target_embeddings_norm = F.normalize(image_batch, p=2, dim=1)
            sim_matrix = pred_embeddings_norm @ target_embeddings_norm.T
            
            correct_scores = torch.diag(sim_matrix).unsqueeze(1)
            ranks = (sim_matrix >= correct_scores).sum(dim=1).float()
            reciprocal_ranks = 1.0 / ranks
            all_reciprocal_ranks.append(reciprocal_ranks)
            
    mrr = torch.cat(all_reciprocal_ranks).mean().item()
    return mrr

# --- 5D. MIXUP (Unchanged) ---
def mixup_data(x, y, alpha=0.2):
    if alpha > 0: lam = np.random.beta(alpha, alpha)
    else: lam = 1
    batch_size = x.size(0)
    indices = torch.randperm(batch_size, device=x.device)
    mixed_x = lam * x + (1 - lam) * x[indices, :]
    mixed_y = lam * y + (1 - lam) * y[indices, :]
    return mixed_x, mixed_y

# Main training

In [ ]:
# --- 16. FINAL Main Training Script (MODIFIED for 1-to-1 Clean Data) ---
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import copy
import itertools
import numpy as np
import json

# --- Store MRR for each fold ---
all_fold_mrr = []

# =====================================================================================
#  K-FOLD OUTER LOOP (Simplified for 1-to-1 data)
# =====================================================================================
# kfold and pair_indices come from Cell 10
for fold, (train_pair_indices, val_pair_indices) in enumerate(kfold.split(pair_indices)):
    print(f"\n{'='*50}")
    print(f"--- STARTING FOLD {fold+1}/{CONFIG['N_FOLDS']} ---")
    print(f"{'='*50}")

    # --- 1. Create Datasets & DataLoaders for this fold ---
    
    # Simple indexing using the pair indices
    train_text_emb = all_text_embeddings[train_pair_indices]
    train_image_emb = all_image_embeddings[train_pair_indices]
    
    val_text_emb = all_text_embeddings[val_pair_indices]
    val_image_emb = all_image_embeddings[val_pair_indices]

    # Create simple 1-to-1 TensorDatasets
    train_dataset = TensorDataset(train_text_emb, train_image_emb)
    val_dataset = TensorDataset(val_text_emb, val_image_emb)

    # --- Use the seeded DataLoaders ---
    train_loader = DataLoader(
        train_dataset, 
        batch_size=CONFIG['BATCH_SIZE'], 
        shuffle=True,
        worker_init_fn=seed_worker, 
        generator=g                 
    )
    val_loader = DataLoader(val_dataset, batch_size=CONFIG['BATCH_SIZE'])

    print(f"Fold {fold+1}: Train size: {len(train_dataset)}, Val size: {len(val_dataset)}")

    # (Optional) We don't need to save split info, as it's just indices

    # --- 2. Re-initialize Models, Losses, and Optimizer ---
    translator_model = TranslatorMLP(
        in_features=all_text_embeddings.shape[1], 
        out_features=all_image_embeddings.shape[1], 
        hidden_features=CONFIG['HIDDEN_FEATURES'],
        num_blocks=CONFIG['NUM_BLOCKS'],
        dropout_rate=CONFIG['DROPOUT_RATE']
    ).to(CONFIG['DEVICE'])

    criterion_triplet = TripletLoss(margin=CONFIG['MARGIN'])
    criterion_pure = PureContrastiveLoss(
        temperature=CONFIG['TEMPERATURE'],
        label_smoothing=CONFIG['LABEL_SMOOTHING'] 
    )

    # --- Use Optimizer from CONFIG (e.g., AdamW) ---
    optimizer = optim.AdamW(
        translator_model.parameters(),
        lr=CONFIG['LR'],
        weight_decay=CONFIG['WEIGHT_DECAY']
    )
    
    total_steps = len(train_loader) * CONFIG['EPOCHS'] // CONFIG['ACCUMULATION_STEPS']
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps)
    
    # --- 3. Training Loop for this fold ---
    best_finetune_mrr = -1.0
    patience_counter = 0

    for epoch in range(CONFIG['EPOCHS']):
        translator_model.train()
        total_train_loss = 0
        
        batch_iterator = tqdm(train_loader, desc=f"Fold {fold+1} Epoch {epoch+1}/{CONFIG['EPOCHS']}")
        
        # --- MODIFIED: Loop is now simpler (text, image) ---
        for i, (text_batch, image_batch) in enumerate(batch_iterator):
            text_batch = text_batch.to(CONFIG['DEVICE'])
            image_batch = image_batch.to(CONFIG['DEVICE'])
            
            target_embeddings = F.normalize(image_batch, p=2, dim=1)
            mixed_text, mixed_targets = mixup_data(text_batch, target_embeddings, alpha=CONFIG['MIXUP_ALPHA'])

            pred_embeddings = translator_model(mixed_text)
            
            loss_triplet = criterion_triplet(pred_embeddings, mixed_targets)
            loss_pure = criterion_pure(pred_embeddings, mixed_targets)
            loss = (CONFIG['HYBRID_ALPHA'] * loss_triplet) + ((1 - CONFIG['HYBRID_ALPHA']) * loss_pure)
            
            loss = loss / CONFIG['ACCUMULATION_STEPS']
            loss.backward()

            if (i + 1) % CONFIG['ACCUMULATION_STEPS'] == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

            total_train_loss += loss.item() * CONFIG['ACCUMULATION_STEPS']
            batch_iterator.set_postfix({"Train Loss": f"{loss.item() * CONFIG['ACCUMULATION_STEPS']:.4f}"})
            
        # --- Validation ---
        avg_train_loss = total_train_loss / len(train_loader)
        
        val_loss = validation_fn(
            translator_model, 
            val_loader, 
            criterion_triplet, 
            criterion_pure, 
            CONFIG['HYBRID_ALPHA'], 
            CONFIG['DEVICE']
        )
        val_mrr = calculate_mrr(translator_model, val_loader, CONFIG['DEVICE'])
        
        current_lr = scheduler.get_last_lr()[0]
        print(f"Fold {fold+1} Epoch {epoch+1} -> Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MRR: {val_mrr:.4f} | LR: {current_lr:e}")
        
        if val_mrr > best_finetune_mrr:
            best_finetune_mrr = val_mrr
            
            model_path = CONFIG['MODEL_PATH_TPL'].format(fold)
            torch.save(translator_model.state_dict(), model_path)
            print(f"  🏆 New best MRR for Fold {fold+1}! Saving model to {model_path}...")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= CONFIG['PATIENCE']:
                print(f"Early stopping triggered for Fold {fold+1}.")
                break

    print(f"\n✅ Fold {fold+1} finished. Best validation MRR: {best_finetune_mrr:.4f}")
    all_fold_mrr.append(best_finetune_mrr)

print(f"\n{'='*50}")
print(f"✅ All {CONFIG['N_FOLDS']} folds trained! ")
print(f"Mean Validation MRR: {np.mean(all_fold_mrr):.4f} (Std: {np.std(all_fold_mrr):.4f})")
print(f"{'='*50}")

# Metrics

 # Submission

In [22]:
# --- 8. GENERATE FINAL SUBMISSION FILE (for MLP-Only) ---
import numpy as np
import torch
from torch.utils.data import DataLoader
import pandas as pd
import json
from tqdm import tqdm

print("--- Generating submission.csv using K-Fold MLP models (Averaging) ---")

# --- 1. Load the test data ---
try:
    test_data = np.load("/kaggle/input/test.clean.npz", allow_pickle=True)
    test_text_emb = torch.from_numpy(test_data['captions/embeddings']).float()
    test_caption_ids = test_data['captions/ids']
    test_loader = DataLoader(test_text_emb, batch_size=CONFIG['BATCH_SIZE'])
    print(f"Test data and caption IDs loaded: {len(test_caption_ids)} samples.")
except FileNotFoundError:
    print("ERROR: Please make sure the 'test.clean.npz' file is in the correct directory.")

# --- 2. Loop through all folds to get predictions ---
all_predictions_list = []

for fold in range(CONFIG['N_FOLDS']):
    print(f"\n--- Generating predictions for Fold {fold+1}/{CONFIG['N_FOLDS']} ---")
    
    # --- Define model path for this fold ---
    model_path = CONFIG['MODEL_PATH_TPL'].format(fold)

    # --- MODIFIED: Load only the TranslatorMLP ---
    translator_model = TranslatorMLP(
        in_features=all_text_embeddings.shape[1], # 1024
        out_features=all_image_embeddings.shape[1], # 1536
        hidden_features=CONFIG['HIDDEN_FEATURES'],
        num_blocks=CONFIG['NUM_BLOCKS'],
        dropout_rate=CONFIG['DROPOUT_RATE']
    ).to(CONFIG['DEVICE'])

    try:
        translator_model.load_state_dict(torch.load(model_path))
    except FileNotFoundError:
        print(f"ERROR: Could not find model file for fold {fold}: {model_path}")
        continue 
        
    translator_model.eval()

    # --- Create predictions in batches for this fold ---
    fold_predictions = []
    with torch.no_grad():
        for text_batch in tqdm(test_loader, desc=f"Predicting Fold {fold+1}"):
            text_batch = text_batch.to(CONFIG['DEVICE'])
            
            # --- MODIFIED: Direct MLP flow ---
            pred_batch = translator_model(text_batch)
            
            fold_predictions.append(pred_batch.cpu().numpy())
    
    all_predictions_list.append(np.concatenate(fold_predictions, axis=0))

if not all_predictions_list:
    print("\nNo predictions were generated. Cannot create submission file.")
else:
    # --- 3. Average the predictions from all folds ---
    print(f"\n--- Averaging predictions from {len(all_predictions_list)} folds ---")
    avg_predictions = np.mean(all_predictions_list, axis=0)
    
    # --- 4. Format for CSV submission ---
    embedding_json_list = [json.dumps(embedding.tolist()) for embedding in avg_predictions]

    submission_df = pd.DataFrame({
        'id': test_caption_ids,
        'embedding': embedding_json_list
    })

    # --- 5. Save the DataFrame to a CSV file ---
    submission_df.to_csv("submission.csv", index=False)

    print(f"\n✅ Ensembled submission file 'submission.csv' has been generated successfully!")
    print("Here's a preview of the first 5 rows:")
    print(submission_df.head())

--- Generating submission.csv using K-Fold MLP models (Averaging) ---
ERROR: Please make sure the 'test.clean.npz' file is in the correct directory.

--- Generating predictions for Fold 1/10 ---


NameError: name 'test_loader' is not defined